In [ ]:
import pandas as pd
from keras import layers
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout,MaxPooling1D
from keras.layers import Input, Dense, Dropout, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Embedding, Add
from keras.layers import Conv1D, GlobalAveragePooling1D, AveragePooling2D, MaxPooling2D, MaxPool1D, ZeroPadding1D, GlobalMaxPooling2D, GlobalAveragePooling2D, LSTM, SpatialDropout1D
from keras.models import Sequential, Model
from keras.preprocessing import image
from keras.utils import plot_model
from keras.applications.inception_v3 import InceptionV3
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam 
import tensorflow as tf


In [ ]:
mit_test_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header=None)
mit_train_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv', header=None)

In [ ]:
print(mit_train_data.shape)
print(mit_test_data.shape)

In [ ]:
mit_train_data.head()

In [ ]:
x_train, y_train = mit_train_data.iloc[: , :-1], mit_train_data.iloc[: , -1]

In [ ]:
y_train

In [ ]:
testX, testy = mit_test_data.iloc[: , :-1], mit_test_data.iloc[: , -1]


In [ ]:
mit_test_data

In [ ]:
y_train = to_categorical(y_train)
testy = to_categorical(testy)

In [ ]:
print("X shape=" +str(x_train.shape))
print("y shape=" +str(y_train.shape))

print("testX shape=" +str(testX.shape))
print("testy shape=" +str(testy.shape))

In [ ]:
ann_model = Sequential()
ann_model.add(Dense(64, activation='relu', input_shape=(187,)))
ann_model.add(Dropout(0.2))
ann_model.add(Dense(64, activation='relu'))
ann_model.add(Dropout(0.2))
ann_model.add(Dense(32, activation='relu'))
ann_model.add(Dropout(0.2))
ann_model.add(Dense(32, activation='relu'))
ann_model.add(Dense(5, activation='softmax'))

In [ ]:
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
ann_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
tf.keras.utils.plot_model(ann_model, to_file='ann_model.png', show_shapes=True, show_layer_names=True)

In [ ]:
ann_model.summary()

In [ ]:
history = ann_model.fit(x_train, y_train,batch_size=128,
epochs=200,
verbose=1,
validation_data = (testX, testy),
callbacks = [learning_rate_reduction,es])

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='orange', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='orange',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
model = Sequential()

model.add(Convolution1D(filters=32, kernel_size=5, strides=1 , input_shape=(187,1)))
model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))

model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))

model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))

model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))

model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation='relu'))
model.add(Convolution1D(filters=32, kernel_size=5, strides=1, padding='same' , activation = 'relu'))
model.add(MaxPooling1D(pool_size=5, strides=2))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
tf.keras.utils.plot_model(ann_model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
history = model.fit(x_train, y_train,batch_size=128,
epochs=200,
verbose=1,
validation_data = (testX, testy),
callbacks = [learning_rate_reduction,es])

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='orange', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='orange',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)